In [1]:
pip install --upgrade openai python-dotenv pandas tqdm tenacity

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os
import json
import pandas as pd
from pathlib import Path
from datetime import datetime

from tqdm import tqdm
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

from openai import OpenAI


In [4]:
# Load the .env file
try:
    from dotenv import load_dotenv
    load_dotenv()
except Exception:
    print("python-dotenv not installed or .env missing")

# Fetch the key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print("OPENAI_API_KEY loaded:", "YES" if OPENAI_API_KEY else "NO")


OPENAI_API_KEY loaded: YES


In [5]:
if not OPENAI_API_KEY:
    raise ValueError("❌ API Key not found. Make sure it's in your .env file as OPENAI_API_KEY.")

client = OpenAI(api_key=OPENAI_API_KEY)

# Tiny check call
try:
    resp = client.responses.create(
        model="gpt-4.1-mini",
        input="Hello! Just checking if the API key works."
    )
    print("API Key check successful ✔️")
except Exception as e:
    print("❌ Key failed:", e)


API Key check successful ✔️


In [6]:
# List all available OpenAI models with categories

from openai import OpenAI
import pandas as pd

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

models = client.models.list()

model_data = []

for m in models.data:
    model_id = m.id

    # Categorize based on name patterns
    if "gpt" in model_id.lower() or "o1" in model_id.lower():
        category = "Chat / Reasoning Models"
    elif "embed" in model_id.lower():
        category = "Embedding Models"
    elif "tts" in model_id.lower() or "audio" in model_id.lower():
        category = "Audio / Speech Models"
    elif "vision" in model_id.lower() or "clip" in model_id.lower():
        category = "Vision Models"
    else:
        category = "Other"

    model_data.append({
        "model_id": model_id,
        "category": category
    })

df_models = pd.DataFrame(model_data).sort_values("model_id")
df_models.reset_index(drop=True, inplace=True)

df_models


,model_id,category
0,babbage-002,Other
1,chatgpt-4o-latest,Chat / Reasoning Models
2,chatgpt-image-latest,Chat / Reasoning Models
3,codex-mini-latest,Other
4,dall-e-2,Other
...,...,...
109,tts-1,Audio / Speech Models
110,tts-1-1106,Audio / Speech Models
111,tts-1-hd,Audio / Speech Models
112,tts-1-hd-1106,Audio / Speech Models


In [7]:
# Check if o4-mini is available in YOUR account and set MODEL_ID accordingly

want = "o4-mini"
has_o4_mini = (df_models["model_id"].str.contains(r"\bo4[-_]mini\b", case=False, na=False)).any()

if has_o4_mini:
    o4_row = df_models[df_models["model_id"].str.contains(r"\bo4[-_]mini\b", case=False, na=False)].sort_values("model_id").head(1)
    MODEL_ID = o4_row.iloc[0]["model_id"]
    print(f"✅ Found {MODEL_ID} in your account. Using this model.")
else:
    # sensible fallback if you don't have access
    MODEL_ID = "gpt-4.1-mini"
    print("⚠️ o4-mini not found on this account. Falling back to:", MODEL_ID)

MODEL_ID


✅ Found o4-mini in your account. Using this model.


'o4-mini'

In [8]:
MODEL_ID = "o4-mini"
print("Model selected:", MODEL_ID)


Model selected: o4-mini


In [9]:
from openai import OpenAI
import os

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
assert OPENAI_API_KEY, "❌ Missing OPENAI_API_KEY"

client = OpenAI(api_key=OPENAI_API_KEY)

try:
    resp = client.responses.create(
        model=MODEL_ID,
        input="Quick hello — please reply with 'OK' if you can hear me."
    )
    print("o4-mini check OK ✅")
    print("Sample output:", resp.output_text[:120].replace("\n"," "))
except Exception as e:
    print("❌ Model check failed:", e)


o4-mini check OK ✅
Sample output: OK


In [10]:
def build_user_prompt(title: str, body: str) -> str:
    return f"""
You are a trauma-informed, non-judgmental peer-support assistant who responds to posts where the author is unsure if an experience counts as sexual abuse. 
Your tone must always be gentle and warm.

Your task:
Analyze the situation described by the user. 
Use your judgment to assess whether consent or boundaries may have been violated based ONLY on what they shared.

CRITICAL RULE:
Do NOT directly say “this was not assault” or give a blunt “no.”
If it’s not clearly a violation, respond in a neutral, supportive way and explain consent.

Choose ONE outcome:

1) Clear consent/boundary violation (high confidence from their details):
- Gently name it as a boundary violation/coercion/assault (supportive wording, not harsh labels).
- Affirm they didn’t deserve it and their feelings make sense.

2) Not clearly a violation OR could be consensual (low/medium confidence):
- Start with: “I’m sorry you went through something that left you feeling confused or uneasy.”
- Validate their feelings and uncertainty.
- Give a simple definition of consent: freely chosen, enthusiastic, specific, informed, and can change anytime.
  Add that pressure, guilt, fear, intoxication, or repeated pushing is not consent.

3) Unclear / missing details:
- Say you can’t be fully sure from the details.
- Validate their discomfort/confusion.
- Give the same simple consent definition as above.
- Optionally include 1–2 soft reflection questions (max 2).

Your response must:
- Be 50–120 words long.
- Use clear, simple human language (no clinical or legal jargon).
- Avoid legal or medical advice.

Now respond to the following post:

TITLE: {title}

BODY:
{body}
"""


In [11]:
def generate_reply(title: str, body: str) -> str:
    # Build the full prompt (since Responses API does not accept system=)
    full_prompt = build_user_prompt(title, body)

    # Make the model call
    resp = client.responses.create(
        model=MODEL_ID,
        input=full_prompt,
    )

    # Extract model output text
    return resp.output_text.strip()


In [12]:
TEST_TITLE = "is this sa?"
TEST_BODY = (
    """
    
        okay i feel ridiculous even typing this but i keep spiraling and i need someone to reality-check me.

so i was hanging out with a friend last night at his apartment. we were watching a movie on the couch and i was kinda tired so i leaned back and he ended up sitting closer than usual. at one point he asked “is this okay?” and i said “yeah it’s fine” because i genuinely didn’t mind. he put his arm around the back of the couch (not like around me, just behind me) and my brain immediately went into panic mode like “wait did i accidentally agree to something” even though nothing else happened.

we didn’t kiss, he didn’t touch me in a weird way, he didn’t say anything sexual, and when the movie ended i left like normal. he even texted after like “home safe?” and that was it.

but now i’m stuck replaying it and thinking: did i consent to something i didn’t mean to? was it weird that he asked “is this okay” like that? i’ve had anxiety all day and i feel sick even though logically it sounds like nothing.

am i overthinking this? i just want to make sure i’m not missing some red flag or something.

    """
)

reply = generate_reply(TEST_TITLE, TEST_BODY)
print(reply)


I’m sorry you went through something that left you feeling confused or uneasy. It sounds like he asked first, you said yes, and there was no further contact. Your feelings of unease are valid. Consent means freely chosen, enthusiastic, specific, informed, and can be withdrawn anytime. Pressure, guilt, fear, intoxication, or repeated pushing isn’t consent. It might help to notice what feels comfortable for you and share your boundaries next time, even for small gestures.


In [14]:
TEST_TITLE = "I don’t know if what happened was assault"
TEST_BODY = (
    "I was with someone I trust and things escalated. I didn’t say no, but I also didn’t really want it. "
    "Now I feel confused and guilty calling it assault. Part of me thinks I’m overreacting, but I can’t stop thinking about it."
)

reply = generate_reply(TEST_TITLE, TEST_BODY)
print(reply)


It sounds like your boundary was crossed when things went ahead without your real yes. Consent means a freely chosen, enthusiastic choice—and if you didn’t really want it, that’s a boundary violation. You didn’t deserve to have your feelings overlooked, and it makes sense you’d feel confused or guilty afterward. Your trust in someone doesn’t erase your right to change your mind or to feel safe. It’s okay to name this as something you didn’t agree to, and your feelings in response are completely valid.


In [15]:
TEST_TITLE = "Can I consider myself a survivor?"
TEST_BODY = (
    "I was wondering if I can consider myself part of the survivor community even if my experience wasn’t necessarily forced/violent? My traumatic experience was medical related and I gave consent but I was in a very distressed state when I said yes and they did not stop immediately when I told them to for medical reasons, as a result I do have sexual related trauma from this experience. I would like to call myself a survivor but I’m worried that using the label even to myself would be insensitive or offensive to other people and women who have experienced more violent/nonconsensual situations. "
)

reply = generate_reply(TEST_TITLE, TEST_BODY)
print(reply)


It sounds like your consent wasn’t fully honored and your request to stop was delayed, which is a boundary violation. That can be very distressing, and you absolutely didn’t deserve it. It’s understandable to feel trauma from what happened. Calling yourself a survivor is a valid way to acknowledge your experience and your strength. Survivor communities include anyone whose boundaries were crossed, even in medical settings. You have every right to use that label for your own healing journey.


In [26]:
TEST_TITLE = "Idk if this was SA, please help"
TEST_BODY = (
    """
    
    "soo this happened when i was 9, almost 10 years old. my brother was 13 at the time. he also has autism

keep in mind, i was wearing a nighty-shirt. i was hanging out in his room as i always used to, i was sitting on the bed. i forget what led to this but he pulled up my nightshirt to expose my behind and positioned my underwear to get a clear view of my small ass, then he started laughing/chuckling as he stared, he told me to keep my underwear like that. after all this time, i can still remember exactly how he did it

i didn't think anything of it but went downstairs to tell my parents and demonstrated what he did to me. they got really mad at him and i hadn't known why back then.

i'm kinda forgetting a lot of my childhood memories but this memory is clearer than others. and it makes me feel uncomfortable. especially because around this time, my brother was going through puberty and i caught him pleasing himself a few months later with things like pillows. idk

does this count as sa? or sh (sexual harassment, not the other kind of sh lol)

"

    """
)

reply = generate_reply(TEST_TITLE, TEST_BODY)
print(reply)


It sounds like your brother crossed a clear boundary by lifting your nightshirt and positioning your underwear so he could stare. That wasn’t your fault, and you absolutely didn’t deserve to feel exposed or confused. What he did was a violation of your personal boundaries, and it’s completely natural to feel uncomfortable or unsettled by it—even years later. Your body always belongs to you, and your feelings about this matter are valid and important.
